# Importing Libraries 📖

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# NLP
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Modeling 
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


# Loading Data set 💾

In [2]:
data = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding = 'latin')

In [3]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


# Pre Processing ⭕

In [4]:
data.drop([data.columns[col] for col in [2, 3, 4]], axis = 1, inplace = True)

In [5]:
data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
encoder = LabelEncoder()
data['v1'] = encoder.fit_transform(data['v1'])
class_mapping = {index : label for index, label in enumerate(encoder.classes_)}

In [7]:
class_mapping

{0: 'ham', 1: 'spam'}

In [8]:
data

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


### Stemming of words

In [9]:
def process_mail(mail):
    ps = PorterStemmer()
    
    mail = mail.lower()
    mail = re.sub(r'<[^<>]+>', ' ', mail)
    mail = re.sub(r'[0-9]+', 'number', mail)
    mail = re.sub(r'(http|https)://[^\s]*', 'httpaddr', mail)
    mail = re.sub(r'[^\s]+@[^\s]+', 'emailaddr', mail)
    mail = re.sub(r'[$]+', 'dollar', mail)
    
    words = word_tokenize(mail)

    for i in range(len(words)):
        words[i] = re.sub(r'[^a-zA-Z0-9]', '', words[i])
        words[i] = ps.stem(words[i])
        
    words = [word for word in words if len(word) >= 1]
    
    return words

### Storing vocabularies

In [10]:
def getVocabulary(emails, vocab_length):
    vocabulary = dict()
    
    for i in range(len(emails)):
        emails[i] = process_mail(emails[i])
        for word in emails[i]:
            if word in vocabulary.keys():
                vocabulary[word] += 1
            else:
                vocabulary[word] = 1
                
    vocabulary = sorted(vocabulary.items(), key = lambda x : x[1], reverse = True)
    vocabulary = list(map(lambda x : x[0], vocabulary[0: vocab_length]))
    vocabulary = {index : word for index, word in enumerate(vocabulary)}
    
    return vocabulary

In [11]:
def getKey(dictionary, val):
    for key, value in dictionary.items():
        if value == val:
            return key

In [12]:
def getIndices(email, vocabulary):
    word_indices = set()
    
    for word in email:
        if word in vocabulary.values():
            word_indices.add(getKey(vocabulary, word))
            
    return word_indices

In [13]:
def getFeatureVector(word_indices, vocab_length):
    feature_vec = np.zeros(vocab_length)
    
    for i in word_indices:
        feature_vec[i] = 1
        
    return feature_vec

### Setting length of vocabulary list 

In [14]:
vocab_length = 2000

In [15]:
vocabulary = getVocabulary(data['v2'].to_list(), vocab_length)

emails = data['v2'].to_list()
emails = list(map(lambda x : process_mail(x), emails))

In [16]:
X = list(map(lambda x : getFeatureVector(getIndices(x, vocabulary), vocab_length), emails))
X = pd.DataFrame(np.array(X).astype(np.int16))

# Model Training 🛠️

In [17]:
X

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,1,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5568,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,1,0,1,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
y = data['v1']

In [19]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: v1, Length: 5572, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [21]:
model = SVC()
model.fit(X_train, y_train)

SVC()

# Scores 📈

### Accuracy

In [22]:
model.score(X_test, y_test)

0.9802690582959641

### Percentage of +ve results in dataset

In [23]:
np.sum(y)/len(y)

0.13406317300789664

In [24]:
y_pred = model.predict(X_test)

### F1 score

In [25]:
f1_score(y_test, y_pred)

0.9236111111111112